In [2]:
import os
import shutil
import polars as pl
import glob

In [6]:
#overall = pl.read_csv("alloc-result.csv", separator=';', has_header=True)
#mcts = pl.read_csv("combined_mcts.csv", separator=',', has_header=True)
#nb_alloc = pl.read_csv("nb_alloc.csv", separator=';', has_header=True).with_columns(pl.lit("max").alias("alloc"))
#df = pl.concat([nb_alloc, overall, mcts], how="vertical")
df = pl.read_csv("min_taskset_result.csv", separator=';', has_header=False).rename({'column_1': 'taskset', 'column_2': 'alloc', 'column_3': 'result'})

In [7]:
df

taskset,alloc,result
str,str,i64
"""min_tasksets/61/20.json""","""ff_little_first""",40
"""min_tasksets/61/98.json""","""ff_little_first""",45
"""min_tasksets/61/77.json""","""ff_little_first""",54
"""min_tasksets/61/61.json""","""ff_little_first""",50
"""min_tasksets/61/36.json""","""ff_little_first""",59
…,…,…
"""min_tasksets/25/23.json""","""ff_sma""",0
"""min_tasksets/25/74.json""","""ff_sma""",0
"""min_tasksets/25/62.json""","""ff_sma""",0


In [9]:
def sort_taskset(df):
    return df.with_columns(
        taskset_num1=pl.col("taskset").str.extract(r"alloc_tasksets/(\d+)/\d+\.json$", 1).cast(pl.Int32),
        taskset_num2=pl.col("taskset").str.extract(r"alloc_tasksets/\d+/(\d+)\.json$", 1).cast(pl.Int32)
    ).sort(["taskset_num1", "taskset_num2"]).drop(["taskset_num1", "taskset_num2"])


df_filtered = df.group_by(["taskset", "alloc"]).agg(
    result=pl.col("result").min()
)
sorted_result = sort_taskset(df_filtered.pivot(
    index="taskset",
    on="alloc",
    values="result",
    aggregate_function="first"  # Use "first", "mean", etc., if duplicates persist
))

sorted_result.write_csv("all_taskset_result.csv")
sorted_result

taskset,ff_sma,ff_little_first,ff_lb
str,i64,i64,i64
"""min_tasksets/45/52.json""",36,32,39
"""min_tasksets/61/95.json""",54,46,54
"""min_tasksets/47/71.json""",46,41,46
"""min_tasksets/59/23.json""",39,38,39
"""min_tasksets/47/72.json""",43,35,48
…,…,…,…
"""min_tasksets/61/34.json""",37,30,37
"""min_tasksets/31/4.json""",0,0,4
"""min_tasksets/37/43.json""",18,12,15
